# LoRA
Low-Rank Adaptation

<img src='https://d.pr/i/kWZ843+' width=500/>

대형 언어 모델(LLM)이나 Transformer 계열에서 **전체 파라미터를 직접 미세조정(fine-tuning)하지 않고**, 주요 가중치(주로 Linear 레이어)에 **저차원 행렬(저랭크 행렬)**로 변경만을 학습하는 방법이다.


**LoRA 공식**

1. 기존 Linear 연산: $Y = W X$
2. LoRA 적용: $Y = (W + \Delta W) X$
  - $\Delta W = B A$
  - 여기서 $r \ll d, k$
3. 즉, $\Delta W$는 **저랭크 행렬 곱**으로 근사화한다.



**원리:**
* 기존의 큰 가중치 행렬 $W$ 대신, $W + BA$ 형태로 쓴다.
* 여기서 $B \in \mathbb{R}^{d \times r}$, $A \in \mathbb{R}^{r \times k}$는 **r이 작은 값(저랭크)** $B, A$만 학습, $W$는 동결(freeze)한다.

**설명:**
* $d$: 원래 가중치 $W\in\mathbb R^{d\times k}$의 **행(row) 크기**, 곧 **입력(input) 차원**이다.
* $k$: $W$의 **열(column) 크기**, 곧 **출력(output) 차원**이다.
* $r$: 어댑터 $A\in\mathbb R^{r\times k}, B\in\mathbb R^{d\times r}$의 **내적(rank) 차원**으로, $r\ll\min(d,k)$인 작은 값이다.

* 대문자 ℝ는 “Real numbers”, 곧 **실수 전체의 집합**을 뜻한다.
* ℝᵈ는 d차원 실수 벡터 공간
* ℝ^{d×k}는 d×k 크기의 실수 행렬들이 모인 공간을 의미한다.

* 저차원 행렬 초기화
  * A ∈ ℝ^{r×d}의 각 원소는 평균 0, 분산 σ²를 갖는 정규분포에서 샘플링해 초기화한다.
  * B ∈ ℝ^{d×r}는 전부 0으로 초기화한다.
  * 이렇게 하면 초반에는 B·A = 0 이므로 원래의 W만 동작하고, 학습이 진행되면서 B만 업데이트해 가벼운 파라미터 조정이 가능하다.


**장점:**

* 파라미터 수와 GPU 메모리 사용량 대폭 감소
* 기존 Pretrained Model의 성능을 최대한 보존
* 다양한 작업에 손쉽게 맞춤화(Prompt Tuning, Adapter 등과 유사한 "파인튜닝 경량화" 전략)



* [LoRA 논문 - LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
* 실제 적용은 Hugging Face `peft` 라이브러리에서 훨씬 더 편하게 쓸 수 있다.




In [1]:
import torch

# 입력데이터
X = torch.randn(3, 8)  # (batch_size, in_feature)

# 가중치
W = torch.randn(4, 8) # (out_feature, in_feature)

# r: 저차원 행렬 크기
r = 2
A = torch.randn(r, 8) # (r, in_feature)
B = torch.randn(4, r) # (out_feature, r)

# 순방향 연산
# 1. 기존 출력
Y_normal = X @ W.T # (3, 8) @ (8, 4) -> (3, 4)

# 2. LoRA 적용된 출력
delta_W = B @ A # (4, 2) @ (2, 8) -> (4, 8)
Y_lora = X @ (W + delta_W).T # (3, 8) @ (8, 4) -> (3, 4)

print(Y_normal)  # 기존 모델 선형층 출력 텐서
print(Y_lora)    # LORA(Delta W) 반영 후 출력 텐서

tensor([[ 3.7546, -2.1008,  0.5638, -0.9199],
        [-1.2150,  1.0843, -1.9324,  2.0813],
        [ 1.3128, -1.3836,  3.0943,  1.5675]])
tensor([[ 0.9211, -1.9563,  4.1684, -1.1867],
        [-5.7657,  3.1328,  5.6262,  0.8978],
        [ 1.9991,  9.5809, 12.9361, -2.9406]])


In [ ]:
# LoRA 선형층 간단 구현
import math
import torch
import torch.nn as nn

class LoRALinear(nn.Module):
    """
    Args:
        r (rank): LoRA에서 추가되는 저랭크 행렬의 랭크(차원)이다.
        - "학습 효율성과 파라미터 효율성의 트레이드오프" 조절 역할.
        - 큰 r → 더 정교하게 원본 모델을 보정할 수 있지만, 파라미터 증가
        - 작은 r → 파라미터와 연산량 적지만, 표현력이 제한됨
        - r이 크면 더 많은 파라미터(학습가능한 자유도)를 갖고, r이 작으면 적은 파라미터(더 단순함).
            - r = 4면, LoRA가 추가하는 변화량이 rank-4 선형 변환에 한정됨(압축).

        alpha: LoRA가 추가하는 저랭크 행렬의 출력값을 얼마나 "키울지" 결정하는 스케일링 계수.
        - alpha를 크게 하면 LoRA 변화량이 더 크게 모델에 반영됨
        - alpha를 작게 하면 LoRA 변화량의 영향이 줄어듦
        - 일반적으로 LoRA 논문에서는 $\alpha = r$로 맞추기를 권장하나(=스케일 1배) 실무에서는 일반적으로 4배 사용
        - $\frac{\alpha}{r}$로 나누어서 LoRA의 영향을 정규화함
            - 예시: r=8, alpha=8이면, $\frac{\alpha}{r}=1$ (스케일 1배, 영향력 그대로)
            - 만약 alpha=16, r=8이면 $\frac{16}{8}=2$ (2배로 스케일 업)
    """
    def __init__(self, in_features, out_features, r=2, alpha=2):
        super().__init__()
        self.W = nn.Linear(in_features, out_features, bias=False)
        self.r = r
        self.alpha = alpha

        # LoRA 저차원행렬
        self.A = nn.Parameter(torch.zeros((r, in_features)))
        self.B = nn.Parameter(torch.zeros((out_features, r)))
        # LoRA 저차원행렬 초기화(논문)
        nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))  # 평균 0, 표준편차 a의 제곱
        nn.init.zeros_(self.B)

    def forward(self, x):
        Wx = self.W(x) # (3, 8) -> (3, 4)
        print(Wx)
        deltaW = self.B @ self.A # (4, 2) @ (2, 8) -> (4, 8)
        lora = deltaW @ x.T # (4, 8) @ (8, 3) -> (4, 3)
        lora = lora.T * self.alpha / self.r # 적용비율 가중치 (3, 4)
        print(lora)
        return Wx + lora # (3, 4) + (3, 4)

torch.manual_seed(42)         # 랜덤시드 고정 (재현성)
lora_layer = LoRALinear(8, 4) # 입력 8차원, 출력 4차원 LoRA 선형층
x = torch.randn(3, 8)         # 입력 더미 데이터 (batch=3, in_features=8)
output = lora_layer(x)        # 순전파 적용(원본 출력 + LoRA 보정)

print(output.shape)
print(output)

tensor([[-0.1657, -0.2278, -0.2746, -0.2284],
        [ 0.2578, -0.0518,  0.4027, -0.0450],
        [ 1.6921,  0.3676,  1.4371, -2.2197]], grad_fn=<MmBackward0>)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], grad_fn=<DivBackward0>)
torch.Size([3, 4])
tensor([[-0.1657, -0.2278, -0.2746, -0.2284],
        [ 0.2578, -0.0518,  0.4027, -0.0450],
        [ 1.6921,  0.3676,  1.4371, -2.2197]], grad_fn=<AddBackward0>)


In [7]:
torch.manual_seed(42)
lora_layer = LoRALinear(8, 4, alpha=4)  # alpha = 4 : LoRA 보정 역활 확대
x = torch.randn(3, 8)
output = lora_layer(x)

print(output.shape)
print(output)

tensor([[-0.1657, -0.2278, -0.2746, -0.2284],
        [ 0.2578, -0.0518,  0.4027, -0.0450],
        [ 1.6921,  0.3676,  1.4371, -2.2197]], grad_fn=<MmBackward0>)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], grad_fn=<DivBackward0>)
torch.Size([3, 4])
tensor([[-0.1657, -0.2278, -0.2746, -0.2284],
        [ 0.2578, -0.0518,  0.4027, -0.0450],
        [ 1.6921,  0.3676,  1.4371, -2.2197]], grad_fn=<AddBackward0>)
